# Strands Agents SDK와 Claude 4 인터리브드 씽킹을 사용한 도구로서의 에이전트

이 노트북은 Strands Agents SDK와 Claude 4의 **인터리브드 씽킹** 기능을 사용하여 전문 에이전트와 함께 지능적인 워크플로우를 조율하는 방법을 보여줍니다.

## 인터리브드 씽킹 이해하기

### 인터리브드 씽킹이란?

인터리브드 씽킹은 Claude 4 모델의 새로운 기능으로 모델이 다음을 수행할 수 있게 합니다:

1. **도구 호출 사이에 사고**: 다음 단계를 결정하기 전에 결과를 처리하고 추론합니다
2. **추론과 함께 여러 도구 연결**: 정교한 다단계 결정을 내립니다
3. **전략을 동적으로 적응**: 중간 결과를 기반으로 접근 방식을 변경합니다

### 작동 방식

인터리브드 씽킹이 있는 것과 없는 것으로 구현된 에이전트의 이벤트 루프 사이에는 많은 유사점이 있습니다:
```
쿼리 → LLM이 사고 중 -> LLM이 도구 호출 결정 -> 이벤트 루프가 도구 호출 -> 출력이 LLM으로 다시 전송 -> [ LLM이 더 이상 도구를 호출할 필요가 없을 때까지 계속 - 최종 답변 렌더링 ]
```

인터리브드 씽킹에서 주목할 주요 차이점은 이벤트 루프가 "결정"이 아닌 LLM의 "사고"에 따라 작동한다는 것입니다. 위 루프의 두 번째 링크인 "사고"를 주목하세요. 전통적인 이벤트 루프에서는 사고가 숨겨져 있습니다. LLM이 도구 호출 결정을 렌더링하거나 최종 답변을 생성할 때까지 기다려야 합니다.

인터리브드 씽킹의 경우, LLM이 여전히 두 번째 단계인 "LLM이 사고 중"에 있는 동안 이벤트 루프로 "사고"를 "누출"하고, 이벤트 루프는 LLM이 그것에 대해 "생각"하자마자 도구를 실행하도록 구성됩니다. 이는 LLM이 사고를 마칠 때까지 실제로 첫 번째 "결정"에서 최종 답변을 가지고 있다는 것을 의미합니다.


### 인터리브드 씽킹 활성화

Strands와 Bedrock에서 이 기능을 활성화하려면:
- `temperature=1` 설정 (씽킹이 활성화될 때 필수)
- 베타 헤더 추가: `"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- 추론 예산 구성: `"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## 설정 및 가져오기

In [1]:
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.models import bedrock

bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## 전문 에이전트를 도구로 정의

먼저 Strands `@tool` 데코레이터를 사용하여 네 개의 전문 에이전트를 생성하겠습니다:
- **연구원**: 사실적 정보 수집
- **데이터 분석가**: 정보 처리 및 분석
- **팩트 체커**: 정보 정확성 검증
- **보고서 작성자**: 세련된 최종 문서 생성

In [2]:
# Strands @tool 데코레이터를 사용하여 도구로 구현된 전문 에이전트
@tool
def researcher(query: str) -> str:
    """
    사실적 정보를 수집하는 연구 전문가.
    
    Args:
        query: 조사할 연구 질문 또는 주제
        
    Returns:
        연구 결과 및 출처
    """
    # 집중된 연구 에이전트 생성
    # 참고: 각 호출은 새로운 에이전트 인스턴스를 생성합니다 (상태 없음)
    research_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
        system_prompt="당신은 연구 전문가입니다. 사실적 정보를 수집하고 가능할 때 출처를 인용하세요. 응답을 200단어 이하로 유지하세요.",
        callback_handler=None  # 도구 에이전트에는 스트리밍 없음
    )
    
    # 연구 작업 실행
    result = research_agent(f"연구: {query}")
    return str(result)

In [3]:
@tool
def data_analyst(data: str) -> str:
    """
    정보를 처리하고 분석하는 데이터 분석가.
    
    Args:
        data: 분석할 원시 데이터 또는 연구 결과
        
    Returns:
        통찰력과 패턴이 포함된 분석
    """
    # 분석가 에이전트는 통찰력 추출에 집중
    analysis_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="당신은 데이터 분석가입니다. 핵심 통찰력을 추출하고, 패턴을 식별하며, 분석적 결론을 제공하세요. 실행 가능한 통찰력에 집중하세요.",
        callback_handler=None
    )
    
    # 제공된 데이터 분석
    result = analysis_agent(f"이 데이터를 분석하고 통찰력을 제공하세요: {data}")
    return str(result)

In [4]:
@tool
def fact_checker(information: str) -> str:
    """
    정보 정확성을 검증하는 팩트 체커.
    
    Args:
        information: 검증할 주장 또는 데이터
        
    Returns:
        정확성 평가가 포함된 팩트 체크 결과
    """
    # 검증을 위한 팩트 체킹 에이전트
    fact_check_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="당신은 팩트 체커입니다. 주장을 검증하고, 신뢰성을 평가하며, 신뢰도 수준을 제공하세요. 의심스러운 진술을 식별하세요.",
        callback_handler=None
    )
    
    # 정보 검증
    result = fact_check_agent(f"이 정보를 팩트 체크하세요: {information}")
    return str(result)

In [5]:
@tool
def report_writer(analysis: str) -> str:
    """
    세련된 최종 문서를 생성하는 보고서 작성자.
    
    Args:
        analysis: 분석된 데이터와 통찰력
        
    Returns:
        형식화된 최종 보고서
    """
    # 전문적인 출력을 위한 작성자 에이전트
    writer_agent = Agent(
        system_prompt="당신은 전문 보고서 작성자입니다. 경영진 요약과 실행 가능한 권장사항이 포함된 명확하고 잘 구조화된 보고서를 작성하세요.",
        callback_handler=None
    )
    
    # 보고서 생성
    result = writer_agent(f"다음을 기반으로 전문 보고서를 작성하세요: {analysis}")
    return str(result)

## 인터리브드 씽킹을 사용한 Claude 4 오케스트레이터

이제 인터리브드 씽킹을 사용하여 전문 에이전트를 지능적으로 조정하는 Claude 4 에이전트인 오케스트레이터를 생성합니다.

### 오케스트레이터 작동 방식:

1. 사용자로부터 고수준 작업을 받습니다
2. 어떤 정보가 필요한지 **생각**합니다
3. 초기 데이터를 수집하기 위해 연구원 도구를 호출합니다
4. 연구 결과와 어떤 분석이 필요한지 **생각**합니다
5. 결과를 처리하기 위해 데이터 분석가를 호출합니다
6. 정확성과 검증 필요성에 대해 **생각**합니다
7. 필요한 경우 팩트 체커를 호출할 수 있습니다
8. 결과를 어떻게 제시할지 **생각**합니다
9. 최종 출력을 위해 보고서 작성자를 호출합니다
10. 응답하기 전에 전체 워크플로우에 대해 **반성**합니다

In [6]:
# Strands를 사용한 인터리브드 씽킹이 포함된 Claude 4 오케스트레이터
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # 지능적인 워크플로우 조정을 위한 오케스트레이터 시스템 프롬프트 정의
        self.system_prompt = """당신은 전문 에이전트에 접근할 수 있는 지능적인 워크플로우 오케스트레이터입니다.

        당신의 역할은 다음 전문 에이전트를 사용하여 워크플로우를 지능적으로 조정하는 것입니다:
        - researcher: 모든 주제에 대한 사실적 정보 수집
        - data_analyst: 데이터 분석 및 통찰력 추출
        - fact_checker: 정보의 정확성 검증
        - report_writer: 세련된 최종 보고서 생성

        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """주어진 작업에 대한 지능적인 워크플로우를 실행합니다.
        
        Args:
            task: 완료할 작업
            enable_interleaved_thinking: 인터리브드 씽킹 활성화 여부 (기본값: True)
        
        오케스트레이터는 다음을 수행합니다:
        1. 작업 요구사항 이해
        2. 최선의 접근 방식에 대해 생각
        3. 전문 에이전트 조정
        4. 단계 간 결과에 대해 반성
        5. 포괄적인 출력 생성
        """
        thinking_mode = "인터리브드 씽킹과 함께" if enable_interleaved_thinking else "인터리브드 씽킹 없이"
        print(f"\n{thinking_mode} 지능적인 워크플로우 시작: {task}")
        print("=" * 70)
        
        # Bedrock을 통해 인터리브드 씽킹이 있거나 없는 Claude 4 구성
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # 씽킹이 활성화될 때 1이어야 함
                additional_request_fields={
                    # 인터리브드 씽킹 베타 기능 활성화
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # 추론 매개변수 구성
                    "reasoning_config": {
                        "type": "enabled",  # 씽킹 켜기
                        "budget_tokens": 3000  # 씽킹 토큰 예산
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Claude 4와 전문 도구로 오케스트레이터 에이전트 생성
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[researcher, data_analyst, fact_checker, report_writer]
        )
        
        prompt = f"""지능적인 워크플로우 조정을 사용하여 이 작업을 완료하세요: {task}

        지침:
        1. 이 작업을 수행하기 위해 어떤 정보가 필요한지 신중히 생각하세요
        2. 전문 에이전트를 전략적으로 사용하세요 - 각각 고유한 강점이 있습니다
        3. 각 도구 사용 후 결과를 반성하고 접근 방식을 조정하세요
        4. 포괄적인 결과를 위해 필요에 따라 여러 에이전트를 조정하세요
        5. 적절할 때 팩트 체킹을 통해 정확성을 보장하세요
        6. 모든 측면을 다루는 포괄적인 최종 응답을 제공하세요
        
        기억하세요: 도구 호출 사이의 사고는 더 나은 결정을 내리는 데 도움이 됩니다.
        이를 사용하여 계획하고, 결과를 평가하고, 전략을 조정하세요.
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"워크플로우 실패: {e}"

## 데모 실행

오케스트레이터가 작동하는 모습을 보겠습니다! 사고하고 사고하는 동안 도구를 호출하는 방식을 관찰하세요.


In [7]:
# 오케스트레이터 생성
print("Strands Agents SDK: Claude 4 인터리브드 씽킹 워크플로우 데모")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ 오케스트레이터가 성공적으로 초기화되었습니다!")
except Exception as e:
    print(f"❌ 오케스트레이터 초기화 실패: {e}")

Strands Agents SDK: Claude 4 인터리브드 씽킹 워크플로우 데모
✅ 오케스트레이터가 성공적으로 초기화되었습니다!


In [8]:
# 테스트 케이스로 워크플로우 실행
test_case = "원격 근무가 생산성에 미치는 영향을 분석하고 전략적 권장사항을 제공하세요"

print(f"📋 작업: {test_case}\n")

try:
    result = orchestrator.run_workflow(test_case)
    
    print(f"\n📊 워크플로우 결과:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ 워크플로우 실행 실패: {e}")

📋 작업: 원격 근무가 생산성에 미치는 영향을 분석하고 전략적 권장사항을 제공하세요


인터리브드 씽킹과 함께 지능적인 워크플로우 시작: 원격 근무가 생산성에 미치는 영향을 분석하고 전략적 권장사항을 제공하세요
이 작업은 원격 근무가 생산성에 미치는 영향을 분석하고 전략적 권장사항을 제공하는 것입니다. 

이를 효과적으로 수행하기 위해 다음과 같은 워크플로우를 계획해야 합니다:

1. 먼저 researcher를 사용하여 원격 근무와 생산성에 관한 기본적인 사실적 정보를 수집
2. data_analyst를 사용하여 수집된 데이터를 분석하고 패턴/통찰력 추출
3. fact_checker를 사용하여 주요 정보의 정확성 검증
4. report_writer를 사용하여 최종 보고서 생성

첫 번째 단계로 researcher를 사용하여 원격 근무가 생산성에 미치는 영향에 대한 연구 결과와 데이터를 수집해보겠습니다.원격 근무가 생산성에 미치는 영향을 포괄적으로 분석하기 위해 체계적인 워크플로우를 시작하겠습니다. 먼저 관련 연구 데이터와 사실적 정보를 수집해보겠습니다.
Tool #1: researcher
좋은 기본 데이터를 수집했습니다. 이제 이 정보를 더 깊이 있게 분석하여 패턴과 통찰력을 추출해야 합니다. data_analyst를 사용하여 다양한 각도에서 분석해보겠습니다.이제 수집된 데이터를 심층 분석하여 더 구체적인 통찰력과 패턴을 찾아보겠습니다.
Tool #2: data_analyst
훌륭한 분석을 얻었습니다. 이제 주요 통계와 결론의 정확성을 검증하기 위해 fact_checker를 사용해보겠습니다. 특히 스탠포드 연구의 13% 생산성 향상 같은 핵심 데이터들을 확인해야 합니다.분석 결과가 유용하게 나왔습니다. 이제 핵심 통계와 주장들의 정확성을 검증하여 신뢰성을 확보하겠습니다.
Tool #3: fact_checker
Fact check 결과를 보니 구체적인 수치들이 완전히 정확하지 않을 수 있다는 점을 확인했습니다. 그러나 일반적인 경향은 맞는 것 같습니다. 이제 이

## 인터리브드 씽킹 없이 시도

오케스트레이터를 호출하고 인터리브드 씽킹을 비활성화하여 실험해보세요. 출력의 차이를 관찰하세요.

In [9]:
# 이제 인터리브드 씽킹 없이 같은 작업을 시도해보겠습니다
print("\n" + "="*70)
print("🔄 이제 인터리브드 씽킹 없이 같은 작업을 실행합니다")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 워크플로우 결과 (인터리브드 씽킹 없음):")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ 워크플로우 실행 실패: {e}")


🔄 이제 인터리브드 씽킹 없이 같은 작업을 실행합니다

인터리브드 씽킹 없이 지능적인 워크플로우 시작: 원격 근무가 생산성에 미치는 영향을 분석하고 전략적 권장사항을 제공하세요
원격 근무가 생산성에 미치는 영향을 분석하기 위한 포괄적인 접근법을 계획해보겠습니다. 먼저 최신 연구 데이터와 통계를 수집하는 것부터 시작하겠습니다.
Tool #1: researcher
이제 수집된 데이터를 분석하여 패턴과 통찰력을 도출하겠습니다.
Tool #2: data_analyst
이 분석 결과의 정확성을 검증하겠습니다.
Tool #3: fact_checker
팩트 체크 결과를 바탕으로 더 일반적이고 검증된 정보를 추가로 수집하겠습니다.
Tool #4: researcher
이제 모든 수집된 정보를 통합하여 최종 보고서를 작성하겠습니다.
Tool #5: report_writer
## 종합 분석 및 전략적 권장사항

저는 전문 에이전트들과의 체계적인 협업을 통해 원격 근무가 생산성에 미치는 영향을 포괄적으로 분석했습니다. 

### 주요 발견사항:

1. **이중적 영향**: 원격 근무는 개인 생산성을 향상시키는 동시에 팀 협업을 감소시키는 트레이드오프를 보입니다.

2. **업종별 차이**: 기술 기업은 원격 근무에서 높은 효과를 보이는 반면, 제조업 등 물리적 업무가 필요한 산업은 제한적입니다.

3. **하이브리드 모델의 부상**: 대부분의 조직이 원격과 대면 근무의 장점을 결합한 하이브리드 모델을 선호하고 있습니다.

### 핵심 전략적 권장사항:

1. **맞춤형 접근법**: 업무 특성과 개인 성향을 고려한 유연한 원격 근무 정책 수립
2. **기술 인프라 강화**: 효과적인 협업을 위한 디지털 도구와 플랫폼 투자
3. **관리 방식 전환**: 근무 시간이 아닌 결과 중심의 성과 평가 체계 도입
4. **조직 문화 유지**: 원격 환경에서도 팀 응집력과 소속감을 강화하는 의도적인 노력

이러한 분석과 권장사항은 연구자의 데이터 수집, 데이터 분석가의 통찰력 도출, 팩트 체커